# Cartilage Generation 
This notebook explains a direct method to generate the lower body cartilage surface mesh based on the bone geometries
    

# imports and user-defined properties

In [1]:
import numpy as np
import meshplot as mp
import os
import igl
import time
from pathlib import Path
import sys
import pandas as pd


sys.path.append(str(Path.home()/'Documents'/'Github'/'libhip'))
sys.path.append('../')

import src

INPUT SUBJECT INFORMATION:

- subject_id: choose between m1 -> m11 to select one of the detaset in the repository.
- i_dim, o_dim: the input and output dimension ( 'mm' = millimeters, 'm' = meters )
- o_format: the format you want the files to be save at ( '.obj' , '.stl' )

In [2]:
# subject id
subject_id = 'm1'

# do you want full or implicit models of the cartilage models?
full_model = True

# dimensions 
i_dim  = "mm"     
o_dim  = "mm"

# suffix
i_format = ".obj"
o_format = ".obj"

DIRECTORIES and PATHS:
- main_dir: locate this directory to where the libhip repository is cloned to.
- ftetwild_dir: locate this path to the 'build' folder of ftetwild.
- i_dir:  locate this directory to the input bone surface mesh files.
- o_dir:  locate this directory to where you. want the cleaned and remeshed bone models to be saved at.

In [3]:
ftetwild_dir = Path.home()/ 'Documents'/ 'Github'/ 'fTetWild'/ 'build'
main_dir     = Path('..')

i_dir          = main_dir/ 'model_generation'/ 'preprocessing_output'/ subject_id
o_dir          = main_dir/ 'model_generation'/ 'cargen_output'/ subject_id

anatomical_dir = main_dir/ 'model_generation'/ 'anatomical_info'
subject_name = '/'+ subject_id + '_joint_anatomical.csv'
subject_anatomical_path = str(anatomical_dir)+ subject_name
    
config_name  = subject_id + '_config'
config_path  = str((main_dir/ 'config'/ config_name).with_suffix(".yml"))
config       = src.Config (config_path)

# Remove all files inside output directory if it exists, otherwise create it
if o_dir.is_dir():
    for file in o_dir.iterdir():
        if file.is_file():
            file.unlink()
else:
    o_dir.mkdir(exist_ok=False)
    
if os.path.exists(subject_anatomical_path):
    os.remove(subject_anatomical_path)

In [4]:
df = pd.read_csv (str(anatomical_dir) + '/default_anatomical.csv', encoding = 'utf-8')
df.loc[1, 'Value'] = subject_id
df.to_csv(subject_anatomical_path, index=False)

In [5]:
# bone input paths
input_bone_tag = subject_id + '_remeshed_'

input_sacrum  = input_bone_tag + 'sacrum_' + i_dim
input_lhip    = input_bone_tag + 'lhip_'   + i_dim
input_rhip    = input_bone_tag + 'rhip_'   + i_dim
input_lfemur  = input_bone_tag + 'lfemur_' + i_dim
input_rfemur  = input_bone_tag + 'rfemur_' + i_dim

input_sacrum_path = str((i_dir/ input_sacrum).with_suffix(i_format))
input_lhip_path   = str((i_dir/ input_lhip).with_suffix(i_format))
input_rhip_path   = str((i_dir/ input_rhip).with_suffix(i_format))
input_lfemur_path = str((i_dir/ input_lfemur).with_suffix(i_format))
input_rfemur_path = str((i_dir/ input_rfemur).with_suffix(i_format))


In [6]:
# bone output paths
output_bone_tag = subject_id + '_cg_bone_'

outout_sacrum = output_bone_tag + 'sacrum'
outout_lhip   = output_bone_tag + 'lhip'
outout_rhip   = output_bone_tag + 'rhip' 
outout_lfemur = output_bone_tag + 'lfemur' 
outout_rfemur = output_bone_tag + 'rfemur'

output_sacrum_path = str((o_dir/ outout_sacrum).with_suffix(o_format))
output_lhip_path   = str((o_dir/ outout_lhip).with_suffix(o_format))
output_rhip_path   = str((o_dir/ outout_rhip).with_suffix(o_format))
output_lfemur_path = str((o_dir/ outout_lfemur).with_suffix(o_format))
output_rfemur_path = str((o_dir/ outout_rfemur).with_suffix(o_format))


In [7]:
# joint output paths

if full_model:
    output_joint_tag = subject_id + '_cg_jnt_'
else:
    output_joint_tag = subject_id + '_cg_jnt_implicit_'
    
# sacroiliac joint
lsj = output_joint_tag + 'lsi'
rsj = output_joint_tag + 'rsi'

# pubic joint
pj  = output_joint_tag + 'pj'

# sacroiliac and pubic joint paths
output_lsj_path = str((o_dir/ lsj).with_suffix(o_format))
output_rsj_path = str((o_dir/ rsj).with_suffix(o_format))
output_pj_path  = str((o_dir/ pj).with_suffix(o_format))

# hip joint with gap
lhj_ac_w_gap = output_joint_tag + 'lh_ac_w_gap' 
lhj_fc_w_gap = output_joint_tag + 'lh_fc_w_gap' 
rhj_ac_w_gap = output_joint_tag + 'rh_ac_w_gap' 
rhj_fc_w_gap = output_joint_tag + 'rh_fc_w_gap'  

# hip joint without gap
lhj_ac_wo_gap = output_joint_tag + 'lh_ac_wo_gap' 
lhj_fc_wo_gap = output_joint_tag + 'lh_fc_wo_gap'
rhj_ac_wo_gap = output_joint_tag + 'rh_ac_wo_gap'
rhj_fc_wo_gap = output_joint_tag + 'rh_fc_wo_gap'

# hip joint w/wo gap paths
output_lhj_ac_w_gap_path = str((o_dir/ lhj_ac_w_gap).with_suffix(o_format))
output_rhj_ac_w_gap_path = str((o_dir/ rhj_ac_w_gap).with_suffix(o_format))
output_lhj_fc_w_gap_path = str((o_dir/ lhj_fc_w_gap).with_suffix(o_format))
output_rhj_fc_w_gap_path = str((o_dir/ rhj_fc_w_gap).with_suffix(o_format))

output_lhj_ac_wo_gap_path = str((o_dir/ lhj_ac_wo_gap).with_suffix(o_format))
output_rhj_ac_wo_gap_path = str((o_dir/ rhj_ac_wo_gap).with_suffix(o_format))
output_lhj_fc_wo_gap_path = str((o_dir/ lhj_fc_wo_gap).with_suffix(o_format))
output_rhj_fc_wo_gap_path = str((o_dir/ rhj_fc_wo_gap).with_suffix(o_format))


# implementation

In [8]:
s_vertices, s_faces  = src.read (input_sacrum_path, i_dim) 
lh_vertices,lh_faces = src.read (input_lhip_path, i_dim)
rh_vertices,rh_faces = src.read (input_rhip_path, i_dim) 
lf_vertices,lf_faces = src.read (input_lfemur_path, i_dim)
rf_vertices,rf_faces = src.read (input_rfemur_path, i_dim) 

number of faces after reading 46456
number of faces after reading 35726
number of faces after reading 35528
number of faces after reading 24908
number of faces after reading 25108


In [9]:
frame = mp.plot(s_vertices, s_faces, c = src.bone, shading = src.sh_true)
frame.add_mesh (lh_vertices, lh_faces, c = src.bone, shading = src.sh_true)
frame.add_mesh (rh_vertices, rh_faces, c = src.bone, shading = src.sh_true)
frame.add_mesh (lf_vertices, lf_faces, c = src.bone, shading = src.sh_true)
frame.add_mesh (rf_vertices, rf_faces, c = src.bone, shading = src.sh_true)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-4.006748…

4

## timer

In [10]:
# start the timer
then = time.time()

# left side cartilages

### left hip joint - acetabular cartilage (with  and without gap)

In [11]:
config_lhj_ac = config.lhj_ac_var

if not full_model:
    config_lhj_ac.full_model = False
    

In [12]:
smooth_lh_vertices, lhj_ac_vertices_wg, lhj_ac_faces_wg, lhj_ac_vertices_wog, lhj_ac_faces_wog, lfc_face_idxs = src.get_hj_ac(lh_vertices, lh_faces,
                                                                                                                              lf_vertices, lf_faces,
                                                                                                                              config_lhj_ac, subject_anatomical_path)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(71.969868…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(71.969868…

minimum thickness in the initial layer with gap 0.71221
minimum thickness in the initial layer without gap 0.79134


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(71.969868…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(71.969868…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(81.457582…

minimum cartilage thickness w/wo gap is:  0.48 / 0.53


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(103.93436…

### left hip joint - femoral cartilage (with gap and without gap)

In [13]:
config_lhj_fc = config.lhj_fc_var

if not full_model:
    config_lhj_fc.full_model = False

In [14]:
# make it
smooth_lf_vertices, lhj_fc_vertices_wg, lhj_fc_faces_wg, lhj_fc_vertices_wog, lhj_fc_faces_wog = src.get_hj_fc (lf_vertices, lf_faces, lh_vertices, lh_faces, lfc_face_idxs, config_lhj_fc, subject_anatomical_path)



Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(103.93436…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(103.93436…

minimum thickness in the initial layer with gap 0.70606
minimum thickness in the initial layer without gap 0.78451


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(103.93436…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(103.93436…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(85.053302…

### measure the gap between the two HJ cartilages

In [15]:
sd_value, _, closest_points = igl.signed_distance(lhj_fc_vertices_wg, lhj_ac_vertices_wg, lhj_ac_faces_wg, return_normals=False)

# print('minimum distance in the left hip joint',np.round(np.min(sd_value),3))

# show the penetration
index = np.where (sd_value < 0 )[0]

frame = mp.plot(lhj_ac_vertices_wg, lhj_ac_faces_wg, c=src.pastel_orange, shading = src.sh_true)
frame.add_mesh (lhj_fc_vertices_wg, lhj_fc_faces_wg, c = src.pastel_yellow, shading = src.sh_true)

if len(index)!= 0 :
    frame.add_points(lhj_fc_vertices_wg[index], shading={"point_color": "green", "point_size": 20})
    

df = pd.read_csv(str(subject_anatomical_path), encoding='utf-8')
df.loc[10, 'Value'] = np.round(np.min(sd_value),3)

df.to_csv(str(subject_anatomical_path), index=False)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(81.457582…

### remove penetration in the "without gap" version

In [16]:
lhj_fc_vertices_wog = src.remove_penetration(lhj_fc_vertices_wog, lhj_ac_vertices_wog, lhj_ac_faces_wog)

### export results

In [17]:
src.save_surface (lhj_fc_vertices_wg, lhj_fc_faces_wg, o_dim, output_lhj_fc_w_gap_path)
src.save_surface (lhj_ac_vertices_wg, lhj_ac_faces_wg, o_dim, output_lhj_ac_w_gap_path)

src.save_surface (lhj_fc_vertices_wog, lhj_fc_faces_wog, o_dim, output_lhj_fc_wo_gap_path)
src.save_surface (lhj_ac_vertices_wog, lhj_ac_faces_wog, o_dim, output_lhj_ac_wo_gap_path)


### left SI joint

In [18]:
config_lsj = config.lsj_var

if not full_model:
    config_lsj.full_model = False

In [19]:
smooth_s_vertices, smooth_lh_vertices, lsj_vertices, lsj_faces = src.get_sj(s_vertices, s_faces,
                                                                           smooth_lh_vertices, lh_faces,
                                                                           config_lsj, subject_anatomical_path)

# export results
src.save_surface (lsj_vertices, lsj_faces, o_dim, output_lsj_path)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-4.006748…

Primary side: max dihedral angle before smoothing is  0.84 radians ( 48.13 degrees).
Primary side: max dihedral angle after smoothing is  0.85 radians ( 48.7 degrees).

Quality control results for the primary side: 
Everything is clean in the primary interface. We will now continue to the secondary interface:



Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-4.022504…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(71.969868…

Secondary layer: max dihedral angle before smoothing is  0.78 radians ( 44.69 degrees).
Secondary layer: max dihedral angle after smoothing is  0.78 radians ( 44.69 degrees).
Quality control results for the base layer: 

Everything is clean in the base layer. We will now continue to create the wall:



Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(71.969868…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-4.022504…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(71.969868…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(42.704328…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(42.704328…


## right side cartilages

### right hip joint - acetabular cartilage (with and without gap)

In [20]:
config_rhj_ac = config.rhj_ac_var

if not full_model:
    config_rhj_ac.full_model = False

In [21]:
smooth_rh_vertices, rhj_ac_vertices_wg, rhj_ac_faces_wg, rhj_ac_vertices_wog, rhj_ac_faces_wog, rfc_face_idxs = src.get_hj_ac( rh_vertices, rh_faces, rf_vertices, rf_faces, config_rhj_ac, subject_anatomical_path )



Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-74.98215…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-74.98215…

minimum thickness in the initial layer with gap 0.88328
minimum thickness in the initial layer without gap 0.98142


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-74.98215…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-74.98215…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-81.78926…

minimum cartilage thickness w/wo gap is:  0.58 / 0.65


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-103.5955…

### right hip joint - femoral cartilage ( with and without gap )

In [22]:
config_rhj_fc = config.rhj_fc_var

if not full_model:
    config_rhj_fc.full_model = False

In [23]:
# make it
smooth_rf_vertices, rhj_fc_vertices_wg, rhj_fc_faces_wg, rhj_fc_vertices_wog, rhj_fc_faces_wog = src.get_hj_fc (rf_vertices, rf_faces, rh_vertices, rh_faces, rfc_face_idxs, config_rhj_fc, subject_anatomical_path)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-103.5955…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-103.5955…

minimum thickness in the initial layer with gap 0.9172
minimum thickness in the initial layer without gap 1.01912


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-103.5955…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-103.5955…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-85.02232…

### measure the gap between the two HJ cartilages

In [24]:
sd_value, _, closest_points = igl.signed_distance(rhj_fc_vertices_wg, rhj_ac_vertices_wg, rhj_ac_faces_wg, return_normals=False)

print('minimum distance in the left hip joint',np.round(np.min(sd_value),3))

# show the penetration
index = np.where (sd_value < 0 )[0]

frame = mp.plot(rhj_ac_vertices_wg, rhj_ac_faces_wg, c=src.pastel_orange, shading = src.sh_true)
frame.add_mesh (rhj_fc_vertices_wg, rhj_fc_faces_wg, c = src.pastel_yellow, shading = src.sh_true)

if len(index)!= 0 :
    frame.add_points(rhj_fc_vertices_wg[index], shading={"point_color": "green", "point_size": 20})
    

df = pd.read_csv(str(subject_anatomical_path), encoding='utf-8')
df.loc[17, 'Value'] = np.round(np.min(sd_value),3)

df.to_csv(str(subject_anatomical_path), index=False)

minimum distance in the left hip joint 0.167


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-81.78926…

In [25]:
rhj_fc_vertices_wog = src.remove_penetration(rhj_fc_vertices_wog, rhj_ac_vertices_wog, rhj_ac_faces_wog)

src.contact_surface(rhj_ac_vertices_wog, rhj_ac_faces_wog, rhj_fc_vertices_wog, rhj_fc_faces_wog, 0 )

min: -0.0074514240509037525


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-81.78926…

contact surface area is:  44.97


### export results

In [26]:
src.save_surface (rhj_fc_vertices_wg, rhj_fc_faces_wg, o_dim, output_rhj_fc_w_gap_path)
src.save_surface (rhj_ac_vertices_wg, rhj_ac_faces_wg, o_dim, output_rhj_ac_w_gap_path)

src.save_surface (rhj_fc_vertices_wog, rhj_fc_faces_wog, o_dim, output_rhj_fc_wo_gap_path)
src.save_surface (rhj_ac_vertices_wog, rhj_ac_faces_wog, o_dim, output_rhj_ac_wo_gap_path)

### right SI joint

In [27]:
config_rsj = config.rsj_var

if not full_model:
    config_rsj.full_model = False

In [28]:
smooth_s_vertices, smooth_rh_vertices, rsj_vertices, rsj_faces = src.get_sj(s_vertices, s_faces,
                                                                           smooth_rh_vertices, rh_faces,
                                                                           config_rsj, subject_anatomical_path)

# export results
src.save_surface (rsj_vertices, rsj_faces, o_dim, output_rsj_path)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-4.006748…

Primary side: max dihedral angle before smoothing is  0.73 radians ( 41.83 degrees).
Primary side: max dihedral angle after smoothing is  0.7 radians ( 40.11 degrees).

Quality control results for the primary side: 
Everything is clean in the primary interface. We will now continue to the secondary interface:



Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-4.006748…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-74.98215…

Secondary layer: max dihedral angle before smoothing is  0.51 radians ( 29.22 degrees).
Secondary layer: max dihedral angle after smoothing is  0.51 radians ( 29.22 degrees).
Quality control results for the base layer: 

Everything is clean in the base layer. We will now continue to create the wall:



Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-74.98215…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-4.006748…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-74.98215…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-49.44783…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-49.44783…

## pubic cartilage

In [29]:
config_pj = config.pj_var

if not full_model:
    config_pj.full_model = False

In [30]:
# make it
smooth_lh_vertices, smooth_rh_vertices, pj_vertices, pj_faces = src.get_gap_pj(smooth_lh_vertices, lh_faces,
                                                                             smooth_rh_vertices, rh_faces,
                                                                             config_pj, subject_anatomical_path)
  
# export results
src.save_surface ( pj_vertices, pj_faces, o_dim, output_pj_path )

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(71.969868…

Primary side: max dihedral angle before smoothing is  0.37 radians ( 21.2 degrees).
Primary side: max dihedral angle after smoothing is  0.37 radians ( 21.2 degrees).

Quality control results for the primary side: 
Everything is clean in the primary interface. We will now continue to the secondary interface:



Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(71.969868…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-74.98215…

Secondary layer: max dihedral angle before smoothing is  0.42 radians ( 24.06 degrees).
Secondary layer: max dihedral angle after smoothing is  0.43 radians ( 24.64 degrees).
Quality control results for the base layer: 

Everything is clean in the base layer. We will now continue to create the wall:


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-74.98215…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(71.969868…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-74.98215…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.727813…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.727813…

## timer

In [31]:
# stop the time
now = time.time()
print("computation time: ", np.round(now-then, 2), 'seconds,', np.round((now-then)/60, 2), 'minutes')


computation time:  192.82 seconds, 3.21 minutes


# voila!

In [32]:
frame = mp.plot( smooth_s_vertices, s_faces, c = src.bone, shading = src.sh_false )
frame.add_mesh ( smooth_lh_vertices, lh_faces, c = src.bone, shading = src.sh_false )
frame.add_mesh ( smooth_rh_vertices, rh_faces, c = src.bone, shading = src.sh_false )
frame.add_mesh ( smooth_lf_vertices, lf_faces, c = src.bone, shading = src.sh_false )
frame.add_mesh ( smooth_rf_vertices, rf_faces, c = src.bone, shading = src.sh_false )
frame.add_mesh ( lhj_ac_vertices_wg, lhj_ac_faces_wg, c = src.pastel_light_blue, shading = src.sh_false )
frame.add_mesh ( lhj_fc_vertices_wg, lhj_fc_faces_wg, c = src.pastel_light_blue, shading = src.sh_false )
frame.add_mesh ( rhj_ac_vertices_wg, rhj_ac_faces_wg, c = src.pastel_light_blue, shading = src.sh_false )
frame.add_mesh ( rhj_fc_vertices_wg, rhj_fc_faces_wg, c = src.pastel_light_blue, shading = src.sh_false )
frame.add_mesh ( lsj_vertices, lsj_faces, c = src.pastel_light_blue, shading = src.sh_false )
frame.add_mesh ( rsj_vertices, rsj_faces, c = src.pastel_light_blue, shading = src.sh_false )
frame.add_mesh ( pj_vertices, pj_faces, c = src.pastel_light_blue, shading = src.sh_false)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-4.006748…

11

# check self intersection in tetgen

In [33]:
# lsi_cart_test  = str((o_dir/ lsi_cart).with_suffix('.stl'))
# rsi_cart_test  = str((o_dir/ rsi_cart).with_suffix('.stl'))
# pubic_cart_test = str((o_dir/ pubic_cart).with_suffix('.stl'))

# # with gap 
# lpelvic_cart_test = str((o_dir/ lpelvic_cart_w_gap).with_suffix('.stl'))
# rpelvic_cart_test = str((o_dir/ rpelvic_cart_w_gap).with_suffix('.stl'))

# lfemoral_cart_test = str((o_dir/ lfemoral_cart_w_gap).with_suffix('.stl'))
# rfemoral_cart_test = str((o_dir/ rfemoral_cart_w_gap).with_suffix('.stl'))

# # withoutgap
# lpelvic_cart_test_wo_gap = str((o_dir/ lpelvic_cart_wo_gap).with_suffix('.stl'))
# rpelvic_cart_test_wo_gap = str((o_dir/ rpelvic_cart_wo_gap).with_suffix('.stl'))

# lfemoral_cart_test_wo_gap = str((o_dir/ lfemoral_cart_wo_gap).with_suffix('.stl'))
# rfemoral_cart_test_wo_gap = str((o_dir/ rfemoral_cart_wo_gap).with_suffix('.stl'))



# src.save_surface ( s1_lcart_vertices, s1_lcart_faces,o_dim, lsi_cart_test )
# src.save_surface ( s1_rcart_vertices, s1_rcart_faces, o_dim, rsi_cart_test )
# src.save_surface ( s2_cart_vertices_w_gap, s2_cart_faces_w_gap, o_dim, lpelvic_cart_test )
# src.save_surface ( s3_cart_vertices_w_gap, s3_cart_faces_w_gap, o_dim, rpelvic_cart_test )
# src.save_surface ( s4_cart_vertices_w_gap, s4_cart_faces_w_gap, o_dim, lfemoral_cart_test )
# src.save_surface ( s5_cart_vertices_w_gap, s5_cart_faces_w_gap, o_dim, rfemoral_cart_test )

# src.save_surface ( s2_cart_vertices_wo_gap, s2_cart_faces_wo_gap, o_dim, lpelvic_cart_test_wo_gap )
# src.save_surface ( s3_cart_vertices_wo_gap, s3_cart_faces_wo_gap, o_dim, rpelvic_cart_test_wo_gap )
# src.save_surface ( s4_cart_vertices_wo_gap, s4_cart_faces_wo_gap, o_dim, lfemoral_cart_test_wo_gap )
# src.save_surface ( s5_cart_vertices_wo_gap, s5_cart_faces_wo_gap, o_dim, rfemoral_cart_test_wo_gap )

# src.save_surface ( p_vertices, p_faces, o_dim, pubic_cart_test )

# os.system ( 'tetgen -d '+ lsi_cart_test )
# os.system ( 'tetgen -d '+ rsi_cart_test )
# os.system ( 'tetgen -d '+ lpelvic_cart_test )
# os.system ( 'tetgen -d '+ rpelvic_cart_test )
# os.system ( 'tetgen -d '+ lfemoral_cart_test )
# os.system ( 'tetgen -d '+ rfemoral_cart_test )
# os.system ( 'tetgen -d '+ pubic_cart_test )


# os.system ( 'tetgen -d '+ lpelvic_cart_test_wo_gap )
# os.system ( 'tetgen -d '+ rpelvic_cart_test_wo_gap )
# os.system ( 'tetgen -d '+ lfemoral_cart_test_wo_gap )
# os.system ( 'tetgen -d '+ rfemoral_cart_test_wo_gap )


## export bones 

In [34]:
src.save_surface ( smooth_s_vertices, s_faces, o_dim, output_sacrum_path ) 
src.save_surface ( smooth_lh_vertices, lh_faces, o_dim, output_lhip_path )
src.save_surface ( smooth_rh_vertices, rh_faces, o_dim, output_rhip_path )
src.save_surface ( smooth_lf_vertices, lf_faces, o_dim, output_lfemur_path )
src.save_surface ( smooth_rf_vertices, rf_faces, o_dim, output_rfemur_path )